In [2]:
%load_ext nb_black
%load_ext Cython

<IPython.core.display.Javascript object>

In [3]:
%%cython

from pyhpfw import ParallelCollector
import glob
import pickle
import numpy as np
import os
import librosa
import soundfile as sf
cimport numpy as np
cimport cython
from libc.stdint cimport uint64_t

cdef extern int __builtin_popcountll(unsigned long long) nogil
        
@cython.boundscheck(False)
@cython.wraparound(False)
cdef unsigned long long _xor(uint64_t[:] array1, uint64_t[:] array2) nogil:
    cdef int i
    cdef unsigned long long cnt = 0
    for i in xrange(len(array1)):
        cnt += __builtin_popcountll(array1[i] ^ array2[i])
    return cnt

@cython.boundscheck(False)
@cython.wraparound(False)
cdef unsigned long long _match_to_ref(uint64_t[:] query, uint64_t[:] ref) nogil:
    cdef unsigned long long bestDistance = 4294967294
    cdef int bestIndex = 0
    cdef int k = len(query)
    cdef int n = len(ref)
    
    cdef int startCol
    cdef int endCol
    cdef unsigned long long distance
    for i in range(n - k + 1):
        startCol = i
        endCol = startCol + k - 1
        distance = _xor(query, ref[startCol : endCol + 1])

        if distance < bestDistance:
            bestDistance = distance
            bestIndex = i

    return bestDistance

def match_to_ref(query, ref):
    return _match_to_ref(query, ref)

def fast_xor(array1, array2):
    return _xor(array1, array2)

<IPython.core.display.Javascript object>

In [4]:
collector = ParallelCollector()

<IPython.core.display.Javascript object>

In [5]:
list_1_path = "/Users/dirceusilva/Documentos/BaseDados/covers32k/list1.list"
list_2_path = "/Users/dirceusilva/Documentos/BaseDados/covers32k/list2.list"

root = "/Users/dirceusilva/Documentos/BaseDados/covers32k"
saved_hashfile = (
    "/Users/dirceusilva/Documentos/development/src/ECAD/sber-hp/results/dump"
)

slices_dir = "/Users/dirceusilva/Documentos/BaseDados/covers32k_slices"
ref_dir = "/Users/dirceusilva/Documentos/BaseDados/covers32k_ref"

def read_list(path, root=None, ext="wav"):
    lista = list(np.loadtxt(path, dtype=str))
    files = []
    if root is not None:
        for file in lista:
            files.append(os.path.join(root, file + "." + ext))
    else:
        files = lista

    return files

<IPython.core.display.Javascript object>

In [6]:
ref_list = read_list(list_1_path, root=root)
tst_list = read_list(list_2_path, root=root)


<IPython.core.display.Javascript object>

In [ ]:
def create_ref_files(inpath, outpath):
    
    

In [ ]:
slice_list = list_1_path
ref_list = list_1_path
filenames_ref = ref_list
filenames_tst = tst_list

filenames_slices = read_list(slice_list, root=slices_dir, ext="wav")
filenames_ref = read_list(slice_list, root=slices_dir, ext="wav")

In [ ]:

# create slices
win = 10.0 # sec

for file_in, file_out in zip(filenames_tst, filenames_slices):
    print(file_in, file_out)
    y,fs = librosa.load(file_in, mono=True)
    winsize = int(fs*win)
    
    start = np.random.randint(0,high=len(y)-winsize)
    end = start + winsize
    
    signal = y[start:end]
    head, tail = os.path.split(file_out)
    if not os.path.exists(head):
        os.makedirs(head)
        
    sf.write(file_out, signal, fs, 'PCM_16')

In [7]:
%%time

hashprints = collector.prepare(filenames_ref)

CPU times: user 55min 28s, sys: 2min 13s, total: 57min 41s
Wall time: 4min 28s


<IPython.core.display.Javascript object>

In [8]:
with open(saved_hashfile, "wb") as fp:
    pickle.dump(hashprints, fp)

<IPython.core.display.Javascript object>

In [9]:
hashprints = []
with open(saved_hashfile, "rb") as fp:
    hashprints = pickle.load(fp)

<IPython.core.display.Javascript object>

In [7]:
filenames_tst

['/Users/dirceusilva/Documentos/BaseDados/covers32k/A_Whiter_Shade_Of_Pale/procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of_Pale.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k/Abracadabra/sugar_ray+14_59+11-Abracadabra.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k/Addicted_To_Love/tina_turner+Tina_Live_In_Europe_CD_1_+09-Addicted_To_Love.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k/All_Along_The_Watchtower/jimi_hendrix_experience+Electric_Ladyland+15-All_Along_The_Watchtower.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k/All_Tomorrow_s_Parties/velvet_underground+Andy_Warhol_with_Nico_+06-All_Tomorrow_s_Parties.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k/America/simon_and_garfunkel+Bookends+03-America.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k/Before_You_Accuse_Me/eric_clapton+Unplugged+02-Before_You_Accuse_Me.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k/Between_The_Bars/glen_phillips+Capitol_Garage+3

<IPython.core.display.Javascript object>

In [10]:
import soundfile as sf
# create slices
win = 10.0 # sec

for file_in, file_out in zip(filenames_tst, filenames_slices):
    print(file_in, file_out)
    y,fs = librosa.load(file_in, mono=True)
    winsize = int(fs*win)
    
    start = np.random.randint(0,high=len(y)-winsize)
    end = start + winsize
    
    signal = y[start:end]
    head, tail = os.path.split(file_out)
    if not os.path.exists(head):
        os.makedirs(head)
        
    sf.write(file_out, signal, fs, 'PCM_16')


/Users/dirceusilva/Documentos/BaseDados/covers32k/A_Whiter_Shade_Of_Pale/annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/A_Whiter_Shade_Of_Pale/annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale.wav
/Users/dirceusilva/Documentos/BaseDados/covers32k/Abracadabra/steve_miller_band+Steve_Miller_Band_Live_+09-Abracadabra.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Abracadabra/steve_miller_band+Steve_Miller_Band_Live_+09-Abracadabra.wav
/Users/dirceusilva/Documentos/BaseDados/covers32k/Addicted_To_Love/robert_palmer+Riptide+03-Addicted_To_Love.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Addicted_To_Love/robert_palmer+Riptide+03-Addicted_To_Love.wav
/Users/dirceusilva/Documentos/BaseDados/covers32k/All_Along_The_Watchtower/bob_dylan+Before_The_Flood_Disc_Two_+07-All_Along_the_Watchtower.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/All_Along_The_Watchtower/bob_dylan+Before_Th

/Users/dirceusilva/Documentos/BaseDados/covers32k/Let_It_Be/beatles+Let_It_Be+06-Let_It_Be.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Let_It_Be/beatles+Let_It_Be+06-Let_It_Be.wav
/Users/dirceusilva/Documentos/BaseDados/covers32k/Little_Wing/corrs+Talk_on_Corners_Special_Edition+12-Little_Wing.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Little_Wing/corrs+Talk_on_Corners_Special_Edition+12-Little_Wing.wav
/Users/dirceusilva/Documentos/BaseDados/covers32k/Lodi/creedence_clearwater_revival+Live_in_Europe+08-Lodi.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Lodi/creedence_clearwater_revival+Live_in_Europe+08-Lodi.wav
/Users/dirceusilva/Documentos/BaseDados/covers32k/Love_Hurts/heart+The_Road_Home+07-Love_Hurts.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Love_Hurts/heart+The_Road_Home+07-Love_Hurts.wav
/Users/dirceusilva/Documentos/BaseDados/covers32k/Maggie_s_Farm/bob_dylan+Live_at_Budokan_Disc_1+06-Maggie_s

/Users/dirceusilva/Documentos/BaseDados/covers32k/Time/tom_waits+Rain_Dogs+09-Time.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Time/tom_waits+Rain_Dogs+09-Time.wav
/Users/dirceusilva/Documentos/BaseDados/covers32k/Tomorrow_Never_Knows/beatles+Revolver+14-Tomorrow_Never_Knows.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Tomorrow_Never_Knows/beatles+Revolver+14-Tomorrow_Never_Knows.wav
/Users/dirceusilva/Documentos/BaseDados/covers32k/Toys_In_The_Attic/aerosmith+Toys_In_The_Attic+01-Toys_In_The_Attic.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Toys_In_The_Attic/aerosmith+Toys_In_The_Attic+01-Toys_In_The_Attic.wav
/Users/dirceusilva/Documentos/BaseDados/covers32k/Train_In_Vain/annie_lennox+Medusa+05-Train_In_Vain.wav /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Train_In_Vain/annie_lennox+Medusa+05-Train_In_Vain.wav
/Users/dirceusilva/Documentos/BaseDados/covers32k/Tush/nazareth+_Snaz+16-Tush.wav /Users/dirceusil

<IPython.core.display.Javascript object>

In [11]:
slices = glob.glob(os.path.join(slices_dir, "**/*.wav"))
slices

['/Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Claudette/everly_brothers+The_Fabulous_Style_of+01-Claudette.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/I_Don_t_Want_To_Miss_A_Thing/aerosmith+Armageddon_Original_Soundtrack_+01-I_Don_t_Want_To_Miss_A_Thing.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Happiness_is_a_Warm_Gun/beatles+White_Album_Disc_1+08-Happiness_is_a_Warm_Gun.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Lady/reo_speedwagon+Arch_Allies_Disc_1+04-Lady.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Tomorrow_Never_Knows/beatles+Revolver+14-Tomorrow_Never_Knows.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Night_Time_Is_The_Right_Time/aretha_franklin+Aretha_Now+04-Night_Time_Is_The_Right_Time.wav',
 '/Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Ooby_Dooby/creedence_clearwater_revival+Cosmo_s_Factory+04-Ooby_Dooby.wav',
 '/Users

<IPython.core.display.Javascript object>

In [12]:
collector.load("")

<IPython.core.display.Javascript object>

In [13]:
from collections import defaultdict

def search(label):
    global collector
    queryFeatures = collector.calc_hashprint(label)
    if len(queryFeatures) == 0:
        return ("INVALID QUERY", -1, -1)

    top = []
    print("Finding ", label)
    global hashprints
    k = len(queryFeatures)

    for f in hashprints:
        refFeatures = f[0]
        refLabel = f[1]
        bestDistance = match_to_ref(queryFeatures, refFeatures)
        top.append((bestDistance, refLabel))

    top = sorted(top)
    return (label, top[:10])


<IPython.core.display.Javascript object>

In [11]:
%%time

#import multiprocessing as mp
import multiprocess as mp

pool = mp.Pool()
ans = pool.map(search, slices)
pool.close()
pool.join()

ModuleNotFoundError: No module named 'multiprocess'

<IPython.core.display.Javascript object>

In [14]:
ans = []
for file in slices[:10]:
    ans.append(search(file))

Finding  /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Claudette/everly_brothers+The_Fabulous_Style_of+01-Claudette.wav
Finding  /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/I_Don_t_Want_To_Miss_A_Thing/aerosmith+Armageddon_Original_Soundtrack_+01-I_Don_t_Want_To_Miss_A_Thing.wav
Finding  /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Happiness_is_a_Warm_Gun/beatles+White_Album_Disc_1+08-Happiness_is_a_Warm_Gun.wav
Finding  /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Lady/reo_speedwagon+Arch_Allies_Disc_1+04-Lady.wav
Finding  /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Tomorrow_Never_Knows/beatles+Revolver+14-Tomorrow_Never_Knows.wav
Finding  /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Night_Time_Is_The_Right_Time/aretha_franklin+Aretha_Now+04-Night_Time_Is_The_Right_Time.wav
Finding  /Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/Ooby_Dooby/creedence_clearwater_revival+Cosmo_s

<IPython.core.display.Javascript object>

In [19]:
ans[8]

('/Users/dirceusilva/Documentos/BaseDados/covers32k_slices_ref/We_Can_Work_It_Out/beatles+1+13-We_Can_Work_It_Out.wav',
 [(4803, 'beatles+1+13-We_Can_Work_It_Out'),
  (20310, 'neil_diamond+Hot_August_Night_Disc_1+10-Red_Red_Wine'),
  (20427, 'robert_palmer+Riptide+03-Addicted_To_Love'),
  (20528, 'deep_purple+Nobody_s_Perfect_Disc_2_2_+07-Hush'),
  (20600, 'aretha_franklin+Aretha_Now+04-Night_Time_Is_The_Right_Time'),
  (20635, 'al_green+Anthology+39-Take_Me_To_The_River'),
  (20649, 'fleetwood_mac+Rumours+11-Gold_Dust_Woman'),
  (20659, 'leann_rimes+Sittin_On_Top_Of_The_World+15-Purple_Rain'),
  (20687, 'elliott_smith+Either_Or+04-Between_The_Bars'),
  (20709, 'creedence_clearwater_revival+Live_in_Europe+08-Lodi')])

<IPython.core.display.Javascript object>

In [20]:
from collections import Counter

counter = Counter([a[0].find(a[1][0][1]) != -1 for a in ans])

<IPython.core.display.Javascript object>

In [21]:
1 - counter[False] / counter[True]

1.0

<IPython.core.display.Javascript object>

In [23]:
import librosa

x = np.random.rand(1000, 5)

y = librosa.util.frame(x, frame_length=10, hop_length=5, axis=0)

<IPython.core.display.Javascript object>

In [24]:
print(y.shape)

(199, 10, 5)


<IPython.core.display.Javascript object>

In [25]:
print(x[:10, :])

[[0.47232994 0.42151262 0.10902632 0.55928467 0.81992219]
 [0.03373441 0.26218802 0.66235769 0.31518655 0.02571327]
 [0.40397618 0.52122604 0.9929573  0.23798788 0.40833402]
 [0.27766876 0.4412011  0.05083741 0.45152137 0.11092971]
 [0.19508237 0.38034507 0.20492677 0.87199822 0.22602431]
 [0.23583441 0.21945859 0.01420484 0.28175229 0.23511873]
 [0.80265665 0.23178462 0.71104553 0.17198997 0.44430012]
 [0.56886534 0.34502748 0.47664881 0.33845634 0.19438857]
 [0.43440421 0.10163655 0.91687568 0.46123346 0.35794896]
 [0.91895267 0.04768644 0.31453359 0.56907759 0.6363371 ]]


<IPython.core.display.Javascript object>

In [27]:
y[0, :]

array([[0.47232994, 0.42151262, 0.10902632, 0.55928467, 0.81992219],
       [0.03373441, 0.26218802, 0.66235769, 0.31518655, 0.02571327],
       [0.40397618, 0.52122604, 0.9929573 , 0.23798788, 0.40833402],
       [0.27766876, 0.4412011 , 0.05083741, 0.45152137, 0.11092971],
       [0.19508237, 0.38034507, 0.20492677, 0.87199822, 0.22602431],
       [0.23583441, 0.21945859, 0.01420484, 0.28175229, 0.23511873],
       [0.80265665, 0.23178462, 0.71104553, 0.17198997, 0.44430012],
       [0.56886534, 0.34502748, 0.47664881, 0.33845634, 0.19438857],
       [0.43440421, 0.10163655, 0.91687568, 0.46123346, 0.35794896],
       [0.91895267, 0.04768644, 0.31453359, 0.56907759, 0.6363371 ]])

<IPython.core.display.Javascript object>

In [46]:
import torch


def pytorch_frame(input, frame_length, hop_length, axis=-1):

    x = input.numpy()
    x_new = librosa.util.frame(x, frame_length, hop_length, axis=axis)

    return torch.from_numpy(x_new.copy())

<IPython.core.display.Javascript object>

In [47]:
x = torch.randn(1000, 5)
print(x.stride())
y = pytorch_frame(x, frame_length=10, hop_length=5, axis=0)

(5, 1)


<IPython.core.display.Javascript object>

In [50]:
print(y.size())

torch.Size([199, 10, 5])


<IPython.core.display.Javascript object>

In [54]:
from skimage.measure import block_reduce

x = np.random.rand(20, 4)
x_1 = block_reduce(x, block_size=(5, 1), func=np.mean)
print(x)
print(x_1)

[[0.07274211 0.24927793 0.20696887 0.48743938]
 [0.02681303 0.82081648 0.81260615 0.01980993]
 [0.52373932 0.1624667  0.08829304 0.99925747]
 [0.04166507 0.46588077 0.27789486 0.58503413]
 [0.44572805 0.2432424  0.89033617 0.1785444 ]
 [0.06742982 0.9515661  0.10781026 0.3609887 ]
 [0.29184264 0.8202929  0.29665615 0.29603725]
 [0.22691615 0.5909555  0.46399433 0.54055895]
 [0.6374273  0.33687573 0.1398456  0.46480026]
 [0.32050642 0.44543203 0.21093994 0.97701301]
 [0.30617086 0.48122391 0.81249487 0.27670322]
 [0.49412329 0.90367902 0.60471536 0.87175465]
 [0.63116557 0.24138998 0.68012792 0.2886952 ]
 [0.89875889 0.36710645 0.67106025 0.41076081]
 [0.04340389 0.99839803 0.9053558  0.32410142]
 [0.89399143 0.22278337 0.63229684 0.86154252]
 [0.13776351 0.81836139 0.59182075 0.17924797]
 [0.87084809 0.01596337 0.60583551 0.48269519]
 [0.85364144 0.84849645 0.73926594 0.9300396 ]
 [0.66553214 0.04913827 0.07752172 0.29267447]]
[[0.22213752 0.38833685 0.45521982 0.45401706]
 [0.30882447

<IPython.core.display.Javascript object>

In [53]:
!pip install scikit-image

     |████████████████████████████████| 13.1 MB 1.8 MB/s            
     |████████████████████████████████| 1.9 MB 1.6 MB/s            
     |████████████████████████████████| 178 kB 1.6 MB/s            
     |████████████████████████████████| 4.3 MB 1.9 MB/s            


<IPython.core.display.Javascript object>

In [55]:
x = np.random.rand(20, 1, 4, 5)

<IPython.core.display.Javascript object>

In [58]:
y = np.reshape(x, (4, 5, 1, 4, 5))

<IPython.core.display.Javascript object>

In [59]:
y.shape

(4, 5, 1, 4, 5)

<IPython.core.display.Javascript object>

In [63]:
q, mod = divmod(21, 5)
print(q, mod)

4 1


<IPython.core.display.Javascript object>